In [64]:
import requests
import json

from pathlib import Path
from pprint import pprint
from datetime import datetime

from openai import OpenAI
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [65]:
import os
from dotenv import load_dotenv

In [66]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Import Prompt
create new LLM, insert prompt and start chain

In [110]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()

In [68]:
print(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects.
You can assume that {card1} is played first, therefor has the earlier timestamp.
Use the {rules_db} as context for your decision.
You will get a {question} about two cards and have to make a decision.

You will follow these steps:

1. Give the oracle texts from both cards.
2. Determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
3. Give your judging: The continuous effect of which card is applied in game: {card1} or {card2}?
4. Explain why you decided this way.
5. Quote the rule you used for your decision.

"""


## RAG
Retrieved Augmented Generation
### Loader

In [96]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules_shortened.txt")
rules_doc = loader.load()

### Splitter

In [97]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
print(rules_splits[0])

page_content='611. Continuous Effects\n\n611.1. A continuous effect modifies characteristics of objects, modifies control of objects, or affects players or the rules of the game, for a fixed or indefinite period.\n\n611.2. A continuous effect may be generated by the resolution of a spell or ability.\n\n611.2a A continuous effect generated by the resolution of a spell or ability lasts as long as stated by the spell or ability creating it (such as “until end of turn”). If no duration is stated, it lasts until the end of the game.' metadata={'source': './data/rules_shortened.txt'}


### Vector Store

In [98]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM
- load template

In [99]:
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)


In [100]:
# prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth

Get card information via scryfall.com and delete unnecessary data: 

In [106]:
card1 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)


{'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'color_identity': ['R'], 'keywords': []}


In [107]:
card2 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

{'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'color_identity': [], 'keywords': [], 'produced_mana': ['B'], 'frame_effects': ['legendary']}


In [111]:
# testing Prompt with input_variables as json
prompt_template_test = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Use Chatmodel instead of LLM: more possible token in prompt
- Outputparser to parse message response from Chatmodel to str

In [112]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template_test | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Blood Moon: "Nonbasic lands are Mountains."
- Urborg, Tomb of Yawgmoth: "Each land is a Swamp in addition to its other land types."

2. The effects of these cards are applied in Layer 4 (Type-changing effects) and Layer 5 (Color-changing effects) according to the layer system in Magic: the Gathering.

3. I will use the rule of Overriding of effects (613.9) to determine which continuous effect takes precedence.

4. Judging: The continuous effect of Blood Moon, which turns nonbasic lands into Mountains, will be applied in the game.

5. I decided this way because in the layer system, type-changing effects (Layer 4) take precedence over color-changing effects (Layer 5). Since Blood Moon's effect changes the type of lands, it will override Urborg's effect of changing the color of lands.

6. Rule used: 613.9 - One continuous effect can override another.


- saving the answers as .txt for comparison: 

In [113]:
current_datetime = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
file_name = current_datetime+".txt"

with open(f'./data/answers/{file_name}', 'w') as f:
    f.write(answer)

In [95]:
# cleanup
rules_db.delete_collection()

## Case 2: Opalescence vs. Humility

In [119]:
card1 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'color_identity': ['W'], 'keywords': []}


In [120]:
card2 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card1)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'color_identity': ['W'], 'keywords': []}
{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'color_identity': ['W'], 'keywords': []}


In [121]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [122]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.
- Humility: All creatures lose all abilities and have base power and toughness 1/1.

2. The effects of these cards are applied in the layers system, as described in rule 613.5.

3. I will use the rule of Overriding of effects (613.9) to determine which continuous effect takes precedence.

4. Judging: The continuous effect of the card 'Humility' is applied in the game. All creatures lose all abilities and have base power and toughness 1/1.

5. Explanation: According to the rule of overriding effects (613.9), when two continuous effects are affecting the same object, the effect generated last "wins." Since 'Humility' was played after 'Opalescence', its effect of reducing all creatures' power and toughness to 1/1 will take precedence over Opalescence's effect of turning non-Aura enchantments into creatures with pow

In [123]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/{file_name}.txt', 'w') as f:
    f.write(answer)

In [ ]:
# cleanup
rules_db.delete_collection()

## Old: Card Variables as retriever for loading them in chain

In [ ]:
loader = JSONLoader(file_path='./data/card1.json', jq_schema='.', text_content=False)
card1_loader = loader.load()

pprint(card1_loader)

In [53]:
loader = JSONLoader(file_path='./data/card2.json', jq_schema='.', text_content=False)
card2_loader = loader.load()

pprint(card2_loader)

[Document(page_content='{"name": "Urborg, Tomb of Yawgmoth", "mana_cost": "", "cmc": 0, "type_line": "Legendary Land", "oracle_text": "Each land is a Swamp in addition to its other land types.", "colors": [], "color_identity": [], "keywords": [], "produced_mana": ["B"], "frame_effects": ["legendary"]}', metadata={'source': '/home/pia/Projects/Studium/WiSe23/Semesterprojekt/MtG/data/card2.json', 'seq_num': 1})]


put both loaded cards as retriever

In [54]:
card1_db = Chroma.from_documents(documents=card1_loader, embedding=OpenAIEmbeddings())
retriever_card1 = card1_db.as_retriever()
card2_db = Chroma.from_documents(documents=card1_loader, embedding=OpenAIEmbeddings())
retriever_card2 = card2_db.as_retriever()

'1. The oracle texts of the two cards are:\n\nCard 1:\nEnchanted creature has flying.\n\nCard 2:\nEnchanted creature loses flying.\n\n2. The continuous effect of the second card is applied in the game. According to the rule "613.9. One continuous effect can override another," when two effects are affecting the same creature and they do not depend on each other, the effect that was generated last "wins." In this case, the effect of losing flying was generated last, so it takes precedence over the effect of gaining flying.\n\n3. I decided this way because the rule clearly states that the effect generated last will override the previous effect when there are conflicting continuous effects on the same creature.\n\n4. The rule used for my decision is 613.9: "One continuous effect can override another."'

In [39]:
# chain = ({"card1": retriever_card1, "card2": retriever_card2, "rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | llm | StrOutputParser())
# chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 5369 tokens (5113 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [23]:
# Version with a ChatModel instead of LLM
# Outputparser to parse message from ChatModel to string 

# chain = ({"card1": retriever_card1, "card2": retriever_card2, "rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
# chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")

'1. Card 1: \n   Oracle text: "Target creature gets +4/+4 until end of turn."\n   \n   Card 2: \n   Oracle text: "Creatures you control get +0/+2."\n\n2. Judging: The continuous effect of the second card, "Creatures you control get +0/+2," is applied in the game.\n\n3. Explanation: In Magic: the Gathering, continuous effects are applied in the order they entered the battlefield, with the earlier timestamp taking precedence. Since the second card was played first, its continuous effect of giving creatures +0/+2 will be applied to the creatures, including the Gray Ogre.\n\n4. Rule used: Magic: the Gathering Comprehensive Rules, section 613.7a: "An effect is considered to be the same source of damage for the duration of the turn if it returns the object in the same state or if it returns the object in a different state in which the object would be if the effect was not applied."'

In [ ]:
# cleanup
rules_db.delete_collection()